In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 162kB/s 
     |████████████████████████████████| 92kB 12.2MB/s 
     |████████████████████████████████| 460kB 39.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Hannanum

In [ ]:
data = pd.read_csv('/content/drive/My Drive/곽민지/train.txt', sep="\", engine='python', encoding="cp949", dtype = {'KEDCD':str})

In [ ]:
data.head(10)

,KEDCD,SEQ,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD
0,0007709505,1,"1. 건축자재 도, 소매업.",G46699
1,0007709505,2,1. 부동산업.,G46699
2,0007709505,3,1. 주택사업.,G46699
3,0007709505,4,1. 각 호에관련된 부대사업 일체.,G46699
4,0007866121,1,"1. 농산물의 제조, 유통, 가공, 판매",C10309
5,0007866121,2,1. 농작업의 전부 또는 일부 대행,C10309
6,0007866121,3,1. 농산물 연구사업,C10309
7,0007866121,4,1. 인터넷상품중개업,C10309
8,0007866121,5,1. 전자상거래업,C10309
9,0007866121,6,1. 과실 및 채소 저장처리업,C10309


In [ ]:
data.isnull().sum()

KEDCD                   0
SEQ                     0
BZ_PPOS_ITM_CTT         0
KSIC10_BZC_CD      101395
dtype: int64

In [ ]:
data[data.notnull()]

,KEDCD,SEQ,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD
0,0007709505,1,"1. 건축자재 도, 소매업.",G46699
1,0007709505,2,1. 부동산업.,G46699
2,0007709505,3,1. 주택사업.,G46699
3,0007709505,4,1. 각 호에관련된 부대사업 일체.,G46699
4,0007866121,1,"1. 농산물의 제조, 유통, 가공, 판매",C10309
...,...,...,...,...
17297027,0008348947,3,1. 장례음식 판매업,S96921
17297028,0008348947,4,1. 장의차 운송업,S96921
17297029,0008348947,5,1. 매점 운영업,S96921
17297030,0008348947,6,1. 부동산임대업,S96921


In [ ]:
kedcd = data.KEDCD.unique()
ksic = data.KSIC10_BZC_CD.unique()

In [ ]:
print(len(kedcd))
print(len(ksic))

1640246
1399


In [ ]:
data2 = data[data.notnull()]

In [ ]:
data2.head()

,KEDCD,SEQ,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD
0,0007709505,1,"1. 건축자재 도, 소매업.",G46699
1,0007709505,2,1. 부동산업.,G46699
2,0007709505,3,1. 주택사업.,G46699
3,0007709505,4,1. 각 호에관련된 부대사업 일체.,G46699
4,0007866121,1,"1. 농산물의 제조, 유통, 가공, 판매",C10309


In [ ]:
# 숫자 제거

data2['BZ_PPOS_ITM_CTT'] = data2['BZ_PPOS_ITM_CTT'].str[2:]
data2.head(10)

,KEDCD,SEQ,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD
0,0007709505,1,"건축자재 도, 소매업.",G46699
1,0007709505,2,부동산업.,G46699
2,0007709505,3,주택사업.,G46699
3,0007709505,4,각 호에관련된 부대사업 일체.,G46699
4,0007866121,1,"농산물의 제조, 유통, 가공, 판매",C10309
5,0007866121,2,농작업의 전부 또는 일부 대행,C10309
6,0007866121,3,농산물 연구사업,C10309
7,0007866121,4,인터넷상품중개업,C10309
8,0007866121,5,전자상거래업,C10309
9,0007866121,6,과실 및 채소 저장처리업,C10309


In [ ]:
dic_x = data2.groupby(['KEDCD','KSIC10_BZC_CD'])['BZ_PPOS_ITM_CTT'].apply(lambda x: x.values.tolist()).to_dict()

In [ ]:
list(dic_x.keys())[0][0]

'0000000001'

In [ ]:
df = pd.DataFrame(columns=['KEDCD','BZ_PPOS_ITM_CTT','KSIC10_BZC_CD'])
df['KEDCD'] = list(map(lambda x: x[0],list(dic_x.keys())))
df['KSIC10_BZC_CD'] = list(map(lambda x: x[1],list(dic_x.keys())))
df['BZ_PPOS_ITM_CTT'] = dic_x.values()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
df.head()        

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD
0,0000000001,"[ 창고업, 써비스업, 부동산 임대업, 자동차 부품 제조업, 각호에 부대한 ...",G45211
1,0000000004,"[ 인쇄업, 부동산임대업, 종이가방,종이쇼핑백,종이상자,비닐가방,비닐쇼핑백 도매업...",C17221
2,0000000006,"[ 의류, 피혁류, 완구류(완농품제외), 식품류(제과업) 및 기타, 각종 일용품의...",G47111
3,0000000007,"[ 방식기계기기 제작 및 재료의 판매업 , 방식업 및 그와...",F42311
4,0000000008,"[ 금속 및 비금속광물의 채광, 매광, 자원개발,제련및 판매, 금속 및 비금속...",C24211


In [ ]:
# KSIC10_BZC_CD 분할

df['Main'] = df['KSIC10_BZC_CD'].str[0]
df['Sub'] = df['KSIC10_BZC_CD'].str[1:]
df.head()

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,Main,Sub
0,0000000001,"[ 창고업, 써비스업, 부동산 임대업, 자동차 부품 제조업, 각호에 부대한 ...",G45211,G,45211
1,0000000004,"[ 인쇄업, 부동산임대업, 종이가방,종이쇼핑백,종이상자,비닐가방,비닐쇼핑백 도매업...",C17221,C,17221
2,0000000006,"[ 의류, 피혁류, 완구류(완농품제외), 식품류(제과업) 및 기타, 각종 일용품의...",G47111,G,47111
3,0000000007,"[ 방식기계기기 제작 및 재료의 판매업 , 방식업 및 그와...",F42311,F,42311
4,0000000008,"[ 금속 및 비금속광물의 채광, 매광, 자원개발,제련및 판매, 금속 및 비금속...",C24211,C,24211


In [1]:
contents = []
for i in df.BZ_PPOS_ITM_CTT:
    contents.append(i)
contents

In [2]:
text = []
for i in contents :
    a = re.sub('[^\w\s]', '', str(i))
    text.append(a)
text

In [ ]:
hannanum = Hannanum()

In [ ]:
words = []
for i in text:
    words.append(hannanum.nouns(i))
words